<a href="https://colab.research.google.com/github/minhnhat232k/Bigdata/blob/main/Assignment2_Bigdata_q4_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark==3.0.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.2/204.2 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 8.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612226 sha256=7679417e0278e8dd0962f15aaa7f196020a294607d8628f6832bb01a6abc496b
  Stored in directory: /root/.cache/pip/wheels/19/b0/c8/6cb894117070e130fc44352c2a13f15b6c27e440d04a84fb48
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.types import IntegerType,BooleanType,DateType

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
spark = SparkSession.builder.getOrCreate()
ratingsDF = spark.read.options(header='True', inferSchema='True', delimiter=',').csv("/content/gdrive/MyDrive/Bigdata-Pyspark-Dataset/Assignment2/ratings_small.csv")
moviesDF = spark.read.options(header='True', inferSchema='True', delimiter=',').csv("/content/gdrive/MyDrive/Bigdata-Pyspark-Dataset/Assignment2/movies_small.csv")

Mounted at /content/gdrive


In [4]:
ratingsDF.show(5)
moviesDF.show(5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



In [5]:
moviesDF = moviesDF.withColumn("Year", f.col("title").substr(-6, 6))
moviesDF = moviesDF.withColumn("Year", f.regexp_replace(f.col("Year"), r"\(|\)", ""))
moviesDF.show()

+-------+--------------------+--------------------+----+
|movieId|               title|              genres|Year|
+-------+--------------------+--------------------+----+
|      1|    Toy Story (1995)|Adventure|Animati...|1995|
|      2|      Jumanji (1995)|Adventure|Childre...|1995|
|      3|Grumpier Old Men ...|      Comedy|Romance|1995|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|1995|
|      5|Father of the Bri...|              Comedy|1995|
|      6|         Heat (1995)|Action|Crime|Thri...|1995|
|      7|      Sabrina (1995)|      Comedy|Romance|1995|
|      8| Tom and Huck (1995)|  Adventure|Children|1995|
|      9| Sudden Death (1995)|              Action|1995|
|     10|    GoldenEye (1995)|Action|Adventure|...|1995|
|     11|American Presiden...|Comedy|Drama|Romance|1995|
|     12|Dracula: Dead and...|       Comedy|Horror|1995|
|     13|        Balto (1995)|Adventure|Animati...|1995|
|     14|        Nixon (1995)|               Drama|1995|
|     15|Cutthroat Island ...|A

In [6]:
ratingsDFLeftJoin = ratingsDF.join(moviesDF, on=["movieId"], how="Left")

In [8]:
ratingsDFLeftJoin.sort("movieId").show()

+-------+------+------+----------+----------------+--------------------+----+
|movieId|userId|rating| timestamp|           title|              genres|Year|
+-------+------+------+----------+----------------+--------------------+----+
|      1|    57|   5.0| 965796031|Toy Story (1995)|Adventure|Animati...|1995|
|      1|   119|   3.5|1435942468|Toy Story (1995)|Adventure|Animati...|1995|
|      1|    63|   5.0|1443199669|Toy Story (1995)|Adventure|Animati...|1995|
|      1|    54|   3.0| 830247330|Toy Story (1995)|Adventure|Animati...|1995|
|      1|    64|   4.0|1161520134|Toy Story (1995)|Adventure|Animati...|1995|
|      1|     7|   4.5|1106635946|Toy Story (1995)|Adventure|Animati...|1995|
|      1|    66|   4.0|1104643957|Toy Story (1995)|Adventure|Animati...|1995|
|      1|    17|   4.5|1305696483|Toy Story (1995)|Adventure|Animati...|1995|
|      1|    68|   2.5|1158531426|Toy Story (1995)|Adventure|Animati...|1995|
|      1|    19|   4.0| 965705637|Toy Story (1995)|Adventure|Ani

In [38]:
ratingsDFResult = ratingsDFLeftJoin.groupBy("movieId").agg(f.count("rating"))
ratingsDFResult.show(5)

+-------+-------------+
|movieId|count(rating)|
+-------+-------------+
|   1580|          165|
|   2366|           25|
|   3175|           75|
|   1088|           42|
|  32460|            4|
+-------+-------------+
only showing top 5 rows



In [39]:
ratingsDFResultAvgRating = ratingsDFLeftJoin.groupBy("movieId").agg(f.avg("rating"))
ratingsDFResultAvgRating.show(5)

+-------+-----------------+
|movieId|      avg(rating)|
+-------+-----------------+
|   1580|3.487878787878788|
|   2366|             3.64|
|   3175|             3.58|
|   1088|3.369047619047619|
|  32460|             4.25|
+-------+-----------------+
only showing top 5 rows



In [42]:
q4Result = ratingsDFResult.join(ratingsDFResultAvgRating, on=["movieId"], how="inner")
q4Result.show(5)

+-------+-------------+-----------------+
|movieId|count(rating)|      avg(rating)|
+-------+-------------+-----------------+
|    148|            1|              5.0|
|    471|           40|             3.55|
|    496|            1|              5.0|
|    833|            6|              2.0|
|   1088|           42|3.369047619047619|
+-------+-------------+-----------------+
only showing top 5 rows



In [43]:
q4Result = q4Result.join(moviesDF, on=["movieId"], how="left")
q4Result.show(20)

+-------+-------------+------------------+--------------------+--------------------+----+
|movieId|count(rating)|       avg(rating)|               title|              genres|Year|
+-------+-------------+------------------+--------------------+--------------------+----+
|    148|            1|               5.0|Awfully Big Adven...|               Drama|1995|
|    471|           40|              3.55|Hudsucker Proxy, ...|              Comedy|1994|
|    496|            1|               5.0|What Happened Was...|Comedy|Drama|Roma...|1994|
|    833|            6|               2.0|High School High ...|              Comedy|1996|
|   1088|           42| 3.369047619047619|Dirty Dancing (1987)|Drama|Musical|Rom...|1987|
|   1238|            9| 4.055555555555555|   Local Hero (1983)|              Comedy|1983|
|   1342|           11|               2.5|     Candyman (1992)|     Horror|Thriller|1992|
|   1580|          165| 3.487878787878788|Men in Black (a.k...|Action|Comedy|Sci-Fi|1997|
|   1591| 

In [44]:
q4Result = q4Result.select("Year", "title", "count(rating)", "avg(rating)")

In [45]:
q4Result.show(20)

+----+--------------------+-------------+------------------+
|Year|               title|count(rating)|       avg(rating)|
+----+--------------------+-------------+------------------+
|1995|Awfully Big Adven...|            1|               5.0|
|1994|Hudsucker Proxy, ...|           40|              3.55|
|1994|What Happened Was...|            1|               5.0|
|1996|High School High ...|            6|               2.0|
|1987|Dirty Dancing (1987)|           42| 3.369047619047619|
|1983|   Local Hero (1983)|            9| 4.055555555555555|
|1992|     Candyman (1992)|           11|               2.5|
|1997|Men in Black (a.k...|          165| 3.487878787878788|
|1997|        Spawn (1997)|           26|2.6346153846153846|
|1997|The Devil's Advoc...|           51| 3.411764705882353|
|1997|  Chinese Box (1997)|            2|              3.25|
|1985|Out of Africa (1985)|           15|3.6666666666666665|
|1984|Children of the C...|           11|2.4545454545454546|
|1991|American Tail: Fi.

In [46]:
q4Result = q4Result.withColumnRenamed("title","Movie_name")
q4Result = q4Result.withColumnRenamed("count(rating)","Num_rating")
q4Result = q4Result.withColumnRenamed("avg(rating)","Average_rating")

In [47]:
q4Result.show(100)

+----+--------------------+----------+------------------+
|Year|          Movie_name|Num_rating|    Average_rating|
+----+--------------------+----------+------------------+
|1995|Awfully Big Adven...|         1|               5.0|
|1994|Hudsucker Proxy, ...|        40|              3.55|
|1994|What Happened Was...|         1|               5.0|
|1996|High School High ...|         6|               2.0|
|1987|Dirty Dancing (1987)|        42| 3.369047619047619|
|1983|   Local Hero (1983)|         9| 4.055555555555555|
|1992|     Candyman (1992)|        11|               2.5|
|1997|Men in Black (a.k...|       165| 3.487878787878788|
|1997|        Spawn (1997)|        26|2.6346153846153846|
|1997|The Devil's Advoc...|        51| 3.411764705882353|
|1997|  Chinese Box (1997)|         2|              3.25|
|1985|Out of Africa (1985)|        15|3.6666666666666665|
|1984|Children of the C...|        11|2.4545454545454546|
|1991|American Tail: Fi...|        10|               2.7|
|1933|    King